<a href="https://colab.research.google.com/github/annechris13/Master-Thesis/blob/master/pdipm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import pandas as pd

In [0]:
#example problem - 2 batches , 2 variable qp
nbatch=2
nBatch=nbatch
nx=2
nineq=2
neq=1
#to do: extract dimensions from problem parameters + check/add batch dimension
Q=torch.tensor([[4,1,1,2],[6,2,2,2]]).view(nbatch,nx,nx).type(torch.DoubleTensor)
p=torch.tensor([[1,1],[1,6]]).view(nbatch,nx).type(torch.DoubleTensor)
G=torch.tensor([[-1,0,0,-1],[-1,0,0,-1]]).view(nbatch,nineq,nx).type(torch.DoubleTensor)
h=torch.tensor([[0,0],[0,0]]).view(nbatch,nineq).type(torch.DoubleTensor)
A=torch.tensor([[1,1],[2,3]]).view(nbatch,neq,nx).type(torch.DoubleTensor)
b=torch.tensor([[1],[4]]).view(nbatch,neq).type(torch.DoubleTensor)

In [0]:
#check if Q is psd:
for i in range(nbatch):
  e,_=torch.eig(Q[i])
  if not torch.all(e[:,0]>0):
    raise RuntimeError("Q is not PSD")

In [0]:
def lu_hack(x):
    data, pivots = x.lu(pivot=not x.is_cuda)
    if x.is_cuda:
        if x.ndimension() == 2:
            pivots = torch.arange(1, 1+x.size(0)).int().cuda()
        elif x.ndimension() == 3:
            pivots = torch.arange(
                1, 1+x.size(1),
            ).unsqueeze(0).repeat(x.size(0), 1).int().cuda()
        else:
            assert False
    return (data, pivots)

In [0]:
def bdiag(d):
    nBatch, sz = d.size()
    D = torch.zeros(nBatch, sz, sz).type_as(d)
    I = torch.eye(sz).repeat(nBatch, 1, 1).type_as(d).bool()
    D[I] = d.squeeze().view(-1)
    return D

In [0]:
def get_Hessian(Q,G,A):
    nbatch,nineq,nx=G.size()
    neq=A.size()[1]
    B1=torch.zeros(nbatch,nx+nineq,nx+nineq).type_as(Q)
    B3=torch.zeros(nbatch,neq+nineq,nx+nineq).type_as(Q)
    B4=torch.zeros(nbatch,neq+nineq,neq+nineq).type_as(Q)

    B1[:,:nx,:nx]=Q
    B1[:,-nineq:,-nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B3[:,:nineq,:nx]=G
    B3[:,-neq:,:nx]=A
    B3[:,:nineq,nineq:]=torch.eye(nineq).repeat(nbatch,1,1).type_as(Q)

    B2=torch.transpose(B3, dim0=2, dim1=1)

    H=torch.cat((torch.cat((B1,B2),dim=2),torch.cat((B3,B4),dim=2)),dim=1)
  
    return H

In [0]:
def solve_kkt(H,rx,rs,rz,ry,d=None):
    if d!=None:
      D=bdiag(d)
      H[:,nx:nx+nineq,nx:nx+nineq]=D
    
    F=torch.cat((rx,rs,rz,ry), dim=1).unsqueeze(2)
    H_lu,H_piv= lu_hack(H)
    step=F.lu_solve(H_lu,H_piv)

    rx=step[:,:nx,:]
    rs=step[:,nx:nx+nineq,:]
    rz=step[:,nx+nineq:-neq,:]
    ry=step[:,-neq:,:]
    return(rx,rs,rz,ry)

In [0]:
def get_step(v, dv):
    a = -v / dv
    a[dv > 0] = max(1.0, a.max())
    return a.min(1)[0].squeeze()

In [88]:
H=get_Hessian(Q,G,A)
A_T=torch.transpose(A,dim0=2,dim1=1)
G_T=torch.transpose(G,dim0=2,dim1=1)
#initial solution
x,s,z,y=solve_kkt(H,-p,torch.zeros(nbatch,nineq).type_as(Q),h,b)
print(x,s,z,y)

tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[-0.3333],
         [-0.6667]],

        [[-0.5294],
         [-0.9804]]], dtype=torch.float64) tensor([[[-3.3333]],

        [[-3.3333]]], dtype=torch.float64)


In [0]:
rx= -(torch.bmm(A_T,y)+torch.bmm(G_T,z)+torch.bmm(Q,x)+p.unsqueeze(2)).squeeze(2)
rs=-z.squeeze(2)
rz=-(torch.bmm(G,x)+s-h.unsqueeze(2)).squeeze(2)
ry=-(torch.bmm(A,x)-b.unsqueeze(2)).squeeze(2)
d=z.squeeze(2)/s.squeeze(2)
dx_aff,ds_aff,dz_aff,dy_aff=solve_kkt(H,rx,rs,rz,ry,d)

In [95]:
alpha = torch.min(torch.min(get_step(z.squeeze(2), dz_aff.squeeze(2)),
                                    get_step(s.squeeze(2), ds_aff.squeeze(2))),
                          torch.ones(nBatch).type_as(Q))
print(alpha)
alpha_nineq = alpha.repeat(nineq, 1).t().unsque
alpha_neq = alpha.repeat(neq, 1).t() if neq > 0 else None
alpha_nx = alpha.repeat(nx, 1).t()
print(x,s,z,y)

tensor([1., 1.], dtype=torch.float64)
tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[0.3333],
         [0.6667]],

        [[0.5294],
         [0.9804]]], dtype=torch.float64) tensor([[[-0.3333],
         [-0.6667]],

        [[-0.5294],
         [-0.9804]]], dtype=torch.float64) tensor([[[-3.3333]],

        [[-3.3333]]], dtype=torch.float64)


In [98]:
#main iterations
max_iter=25
#only affine steps now

for i in range(max_iter):
    rx= -(torch.bmm(A_T,y)+torch.bmm(G_T,z)+torch.bmm(Q,x)+p.unsqueeze(2)).squeeze(2)
    rs=-z.squeeze(2)
    rz=-(torch.bmm(G,x)+s-h.unsqueeze(2)).squeeze(2)
    ry=-(torch.bmm(A,x)-b.unsqueeze(2)).squeeze(2)
    d=z.squeeze(2)/s.squeeze(2)

    #affine step calculation
    dx,ds,dz,dy=solve_kkt(H,rx,rs,rz,ry,d)

    #step size calculation
    alpha = torch.min(torch.min(get_step(z.squeeze(2), dz_aff.squeeze(2)),
                                    get_step(s.squeeze(2), ds_aff.squeeze(2))),
                          torch.ones(nBatch).type_as(Q))
    print(alpha)
    alpha_nineq = alpha.repeat(nineq, 1).t().unsqueeze(2)
    alpha_neq = alpha.repeat(neq, 1).t().unsqueeze(2) if neq > 0 else None
    alpha_nx = alpha.repeat(nx, 1).t().unsqueeze(2)

    #update step
    x+=alpha_nx*dx
    s+=alpha_nineq*ds
    z+=alpha_nineq*dz
    y+=alpha_neq*dy

    print(x)


tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torch.float64)
tensor([[[0.2500],
         [0.7500]],

        [[0.5000],
         [1.0000]]], dtype=torch.float64)
tensor([1., 1.], dtype=torc